In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
from pygom import Transition, TransitionType, DeterministicOde, epi_analysis, SimulateOde
from IPython.display import Image, Markdown

# setting up the plotting
plt.rcParams['figure.figsize'] = [14, 10]
plt.rcParams['font.size'] = 14

matplotlib.rc('font', size=14)
matplotlib.rc('axes', titlesize=14)
%matplotlib inline

SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

x = 1000
Np = 678
parallel = True

In [ ]:
def stoc_plot_quants(simX, simT, Sind, Eind, Iuind, Idind, Mind, Hind, Rpind, Rhind, x,
                     min_case=5, figdir='figures/fig'):
    '''Plot quantiles of solutions.'''
    
    sus = np.array([sim_res[:,Sind] for sim_res in simX])
    exp = np.sum(np.array([sim_res[:,Eind] for sim_res in simX]), axis=2)
    inu = np.sum(np.array([sim_res[:,Iuind] for sim_res in simX]), axis=2)
    ind = np.sum(np.array([sim_res[:,Idind] for sim_res in simX]), axis=2)
    med = np.sum(np.array([sim_res[:,Mind] for sim_res in simX]), axis=2)
    hos = np.sum(np.array([sim_res[:,Hind] for sim_res in simX]), axis=2)
    rpr = np.array([sim_res[:,Rpind] for sim_res in simX])
    rho = np.array([sim_res[:,Rhind] for sim_res in simX])    
    
    
    # condition for removing simulations when extinction occurs
    # returns True False list
    cond = rpr[:,-1] + rho[:,-1] > min_case
    cond = cond[:,0]   

    sus_quant = np.transpose(np.quantile(sus[cond,:], (0.025, 0.25, 0.5, 0.75, 0.975), axis=0))[0]
    exp_quant = np.transpose(np.quantile(exp[cond,:], (0.025, 0.25, 0.5, 0.75, 0.975), axis=0))
    inu_quant = np.transpose(np.quantile(inu[cond,:], (0.025, 0.25, 0.5, 0.75, 0.975), axis=0))
    ind_quant = np.transpose(np.quantile(ind[cond,:], (0.025, 0.25, 0.5, 0.75, 0.975), axis=0))
    med_quant = np.transpose(np.quantile(med[cond,:], (0.025, 0.25, 0.5, 0.75, 0.975), axis=0))
    hos_quant = np.transpose(np.quantile(hos[cond,:], (0.025, 0.25, 0.5, 0.75, 0.975), axis=0))
    rpr_quant = np.transpose(np.quantile(rpr[cond,:], (0.025, 0.25, 0.5, 0.75, 0.975), axis=0))[0]
    rho_quant = np.transpose(np.quantile(rho[cond,:], (0.025, 0.25, 0.5, 0.75, 0.975), axis=0))[0]

    fig = plt.figure(figsize=[18,16])

    ax = plt.subplot(3,3,1)
    ax.plot(simT, sus_quant[:,2], label='median', color='black')
    
    ax.fill_between(simT, sus_quant[:,0], sus_quant[:,4], 
                    color='blue', alpha=0.1, label='2.5 - 97.5 quantile')
    ax.fill_between(simT, sus_quant[:,1], sus_quant[:,3],
                    color='blue', alpha = 0.2, label='25 - 75 quantile')
    
    ax.set_ylabel('People')
    ax.set_title('Susceptible')

    ax = plt.subplot(3,3,2)
    ax.plot(simT, exp_quant[:,2], label='median', color='black')
    
    ax.fill_between(simT, exp_quant[:,0], exp_quant[:,4], 
                    color='blue', alpha = 0.1, label='2.5 - 97.5 quantile')
    ax.fill_between(simT, exp_quant[:,1], exp_quant[:,3], 
                    color='blue', alpha = 0.2, label='25 - 75 quantile')
    
    ax.set_title('Incubating')

    ax = plt.subplot(3,3,3)
    ax.plot(simT, inu_quant[:,2], label='median', color='black')
    
    ax.fill_between(simT, inu_quant[:,0], inu_quant[:,4],
                    color='blue', alpha=0.1, label='2.5 - 97.5 quantile')
    ax.fill_between(simT, inu_quant[:,1], inu_quant[:,3],
                    color='blue', alpha=0.2, label='25 - 75 quantile')
    
    ax.set_xlabel('Days since initial seeding')
    ax.set_ylabel('People')
    ax.set_title('Infectious - not clinical')
    
    ax = plt.subplot(3,3,4)
    ax.plot(simT, ind_quant[:,2], label='median', color='black')
    
    ax.fill_between(simT, ind_quant[:,0], ind_quant[:,4],
                    color='blue', alpha=0.1, label='2.5 - 97.5 quantile')
    ax.fill_between(simT, ind_quant[:,1], ind_quant[:,3],
                    color='blue', alpha=0.2, label='25 - 75 quantile')
    
    ax.set_xlabel('Days since initial seeding')
    ax.set_ylabel('People')
    ax.set_title('Infectious - clinical')

    ax = plt.subplot(3,3,5)
    ax.plot(simT, med_quant[:,2], label='median', color='black')
    
    ax.fill_between(simT, med_quant[:,0], med_quant[:,4],
                    color='blue', alpha=0.1, label='2.5 - 97.5 quantile')
    ax.fill_between(simT, med_quant[:,1], med_quant[:,3],
                    color='blue', alpha=0.2, label='25 - 75 quantile')
    
    ax.legend(loc=2, facecolor='white', frameon=False)
    ax.set_xlabel('Days since initial seeding')
    ax.set_title('Medical isolation')
    
    ax = plt.subplot(3,3,6)
    ax.plot(simT, hos_quant[:,2], label='median', color='black')
    
    ax.fill_between(simT, hos_quant[:,0], hos_quant[:,4],
                    color='blue', alpha=0.1, label='2.5 - 97.5 quantile')
    ax.fill_between(simT, hos_quant[:,1], hos_quant[:,3],
                    color='blue', alpha=0.2, label='25 - 75 quantile')
    ax.legend(loc=2, facecolor='white', frameon=False)
    
    ax.set_xlabel('Days since initial seeding')
    ax.set_title('Hospitalised')
    
    
    ax = plt.subplot(3,3,7)
    ax.plot(simT, rpr_quant[:,2], label='median', color='black')
    
    ax.fill_between(simT, rpr_quant[:,0], rpr_quant[:,4],
                    color='blue', alpha=0.1, label='2.5 - 97.5 quantile')
    ax.fill_between(simT, rpr_quant[:,1], rpr_quant[:,3],
                    color='blue', alpha=0.2, label='25 - 75 quantile')
    ax.legend(loc=2, facecolor='white', frameon=False)
    
    ax.set_xlabel('Days since initial seeding')
    ax.set_title('No longer infectious (non-clinical) and recovered (not hospitalised)')
    
    ax = plt.subplot(3,3,8)
    ax.plot(simT, rho_quant[:,2], label='median', color='black')
    
    ax.fill_between(simT, rho_quant[:,0], rho_quant[:,4],
                    color='blue', alpha=0.1, label='2.5 - 97.5 quantile')
    ax.fill_between(simT, rho_quant[:,1], rho_quant[:,3],
                    color='blue', alpha=0.2, label='25 - 75 quantile')
    ax.legend(loc=2, facecolor='white', frameon=False)
    
    ax.set_xlabel('Days since initial seeding')
    ax.set_title('Recovered from hospital (not returned to prison)')

    plt.savefig(figdir, dpi=300)

    per_cond =(x-rpr[cond,:].shape[0])/rpr.shape[0]*100

    print('%d %% of the %d realisations result in fewer than %d cases. These have not been plotted.' %(per_cond, x, min_case))
    
    return cond

def attack_sizes_stoc(simX, Np):
    '''Print out the infection and hospitalised percentages.'''
    total_infected = (rpr_quant[-1,:]+rho_quant[-1,:])/Np*100
    total_hospitalised = rho_quant[-1,:]/Np*100
    
    x = Markdown("""
    Under these assumptions:
    - {tot}% prisoners will be infected (0.025, 0.25, 0.5, 0.75, 0.975 quantiles)
    - {tothosp}% prisoners will be hospitalised (0.025, 0.25, 0.5, 0.75, 0.975 quantiles)
    """.format(tot=np.round(total_infected,2), tothosp=np.round(total_hospitalised,2)))
    
    return x

# Prison scenario modelling 

Updated after conversation with Tom.

1. Find kc and kh parameters that result in approx:
- 50% population clinically attacked
- 5.77% of infected population hospitalised

2. Plot attack rate against frequecy of simulations for stochastic run.

3. Plot final attack size for each regime change with bounds


### Model
The model is based on an SEIR structure. 
- Susceptible indivdiuals are infected ($\beta I / N_t$)
- Individuals incubate for $t_a$ days
- A proportion($k_c$) of these individuals will be detected (due to symptoms)
   - It takes $t_d$ days for these individuals to be detected
   - A proportion ($k_h$) will enter hospital
       - It takes $t_h$ days for recovery (considered to be the most severe cases)
       - These individuals are not reintegrated back into the prison population
    - The remaining proportion ($1-k_h$) will be medically isolated.
        - It takes $t_m$ days for recovery (considered to be mild illness)
        - These individuals re-enter the community population
- The remaining proportion of infected individuals ($1-k_c$) are not identified (due to not seeking healthcare, not displaying symptoms or having sub-clinical symptoms)
    - They are infectious for $t_g$ days
    - These individuals remain in the prison population.
    


|Parameter |Description                                         |Value           |
|:---------|:---------------------------------------------------|:---------------|
|$\beta$   |Transmission rate                                   |0.5 days$^{-1}$ | 
|$R_0$     |Reproduction number                                 |3               |
|$t_a$     |Incubation period                                   |6 (5-7) days    |
|$t_g$     |Infectious period                                   |4 (2-6) days    |
|$t_d$     |Detection period                                    |1 day           |
|$t_m$     |Medical isolation period                            |7 days          |
|$t_h$     |Hospitalisation period                              |22 days         |
|$k_c$     |Proportion of infected that are clinically attacked |tbd            |
|$k_h$     |Proportion of $k_c$ that are hospitalised           |tbd         |



### Cohorting
This model does not cohort people in the sense of grouping by date of symptom. People are removed after $t_d$ days of symptom onset and medically isolated. They re-enter the prison population after $t_m$ days. 

### Shielding
1.2% of the total population are considered highly vulnerable.
In the intervention scenarios, 1.2% will be removed from the population. 
- If shielding is 100% effective across all sites, then there will be 0 additions to infection and hospitalisation numbers.
- If shielding is 0% effective across all sites, with a 100% attack rate, the total shielded population can be added to the infection number. Presumably given the vulnerable nature of this grop, all would be hospitalised.
- There is insufficient information to make a reasonable estimate about probability of attack, attack rate and effectiveness of shielding inbetween these two extremes.

In [ ]:
# number of compartments to split each state into (Erlang distribution)
ne = 2 # incubating
nu = 2 # undetected infectious
nd = 2 # detected infectious
nm = 2 # medically isolated
nh = 2 # hospitalised

In [ ]:
def prison(ne=2, nu=2, nd=2, nm=2, nh=2):
    '''
    Template for seir compartmental structure which automatically 
    generates additional E and I compartments. 
    '''
    
    states = ['S']                               # susceptible 
    states += ['E%d' %(x+1) for x in range(ne)]  # incubating 
    states += ['Iu%d' %(x+1) for x in range(nu)] # infectious undetected 
    states += ['Id%d' %(x+1) for x in range(nd)] # infectious detected 
    states += ['M%d' %(x+1) for x in range(nm)]  # medical isolation
    states += ['H%d' %(x+1) for x in range(nh)]  # infectious household 
    states += ['Rh']                             # removed from hospital
    states += ['Rm']                             # removed from medical isolation 
    states += ['Rp']                             # undetected no longer infectious 

    # generate output to facilitate summing multiple compartments
    group = ['S']
    group += ['E' for x in range(ne)]
    group += ['Iu' for x in range(nu)]
    group += ['Id' for x in range(nd)]
    group += ['M' for x in range(nm)]
    group += ['H' for x in range(nh)]
    group += ['Rh']
    group += ['Rm']
    group += ['Rp']

    # model parameters
    parameters = ['beta',  # ro = beta/gamma= beta*th2
                  'ta',    # incubation period 
                  'tg',    # infectious period 
                  'td',    # mean time to detection for individauls that will be detected
                  'tm',    # mean time in 
                  'th',    # mean time in hospital
                  'kc',    # proportion of detected cases
                  'kh',    # proportion of hospitalised cases
                  'Np',    # total prison population
                  'ne',    # number of compartments for distributing incubation period 
                  'nu',    # number of compartments for distributing undetected infectious compartments
                  'nd',    # number of compartments for distributing detected infectious compartments
                  'nm',    # number of compartments for distributing medically isolated compartments
                  'nh'     # number of compartments for distributing hospitalised comparments
                 ]

    transitions = []

    ## infections
    # new infections generated from Iu compartments
    for z in range(nu):
        transitions += [Transition(origin='S', destination='E1', 
                        equation='beta*S*Iu%d/(Np-M1-M2-H1-H2-Rh)' %(z+1),
                        transition_type=TransitionType.T)]

    # new infections generated from Id compartments
    for z in range(nd):
        transitions += [Transition(origin='S', destination='E1', 
                        equation='beta*S*Id%d/(Np-M1-M2-H1-H2-Rh)' %(z+1),
                        transition_type=TransitionType.T)]

    ## incubating
    # generate transistions for E states
    if ne > 1:
        for x in range(ne-1):
            transitions += [Transition(origin='E%d' %(x+1), destination='E%d' %(x+2),
                            equation='%d*E%d/ta' %(ne,x+1),
                            transition_type=TransitionType.T)]

    # transition from last E to Iu1        
    transitions += [Transition(origin='E%d' %ne, destination='Iu1', 
                    equation='(1-kc)*%d*E%d/ta' %(ne,ne),
                    transition_type=TransitionType.T)]       

    # transition from last E to Id1  
    transitions += [Transition(origin='E%d' %ne, destination='Id1', 
                    equation='kc*%d*E%d/ta' %(ne,ne),
                    transition_type=TransitionType.T)] 

    ## infectious undetected
    # generate transitions for Iu states
    if nu > 1:
        for y in range(nu-1):
            transitions += [Transition(origin='Iu%d' %(y+1), destination='Iu%d' %(y+2),
                            equation='%d*Iu%d/tg' %(nu,y+1),
                            transition_type=TransitionType.T)]

    # transition from last Iu to Rp        
    transitions += [Transition(origin='Iu%d' %nu, destination='Rp', 
                    equation='%d*Iu%d/tg' %(nu,nu),
                    transition_type=TransitionType.T)]   

    ## infectious detected
    # generate transitions for Id states
    if nd > 1:
        for y in range(nd-1):
            transitions += [Transition(origin='Id%d' %(y+1), destination='Id%d' %(y+2),
                            equation='%d*Id%d/td' %(nd,y+1),
                            transition_type=TransitionType.T)]

    # transition from last Id to M1        
    transitions += [Transition(origin='Id%d' %nd, destination='M1', 
                    equation='(1-kh)*%d*Id%d/td' %(nd,nd),
                    transition_type=TransitionType.T)]   

    # transition from last Id to H1        
    transitions += [Transition(origin='Id%d' %nd, destination='H1', 
                    equation='kh*%d*Id%d/td' %(nd,nd),
                    transition_type=TransitionType.T)]   

    ## medical isolation
    # generate transitions for M states
    if nm > 1:
        for y in range(nm-1):
            transitions += [Transition(origin='M%d' %(y+1), destination='M%d' %(y+2),
                            equation='%d*M%d/tm' %(nm,y+1),
                            transition_type=TransitionType.T)]

    # transition from last M to Rp        
    transitions += [Transition(origin='M%d' %nm, destination='Rm', 
                    equation='%d*M%d/tm' %(nm,nm),
                    transition_type=TransitionType.T)]        

    ## hospital
    # generate transitions for H states
    if nh > 1:
        for y in range(nh-1):
            transitions += [Transition(origin='H%d' %(y+1), destination='H%d' %(y+2),
                            equation='%d*H%d/th' %(nh,y+1),
                            transition_type=TransitionType.T)]

    # transition from last H to Rh        
    transitions += [Transition(origin='H%d' %nh, destination='Rh',
                    equation='%d*H%d/th' %(nh,nh),
                    transition_type=TransitionType.T)]        

    model = DeterministicOde(states, parameters, transition=transitions)
    
    model.parameters = {'ne':ne, 'nu':nu, 'nd':nd, 'nm':nm, 'nh':nh}

    return model, group



def run_prison(model, inE, inIu, inId, inM, inH, inRh,
               inRm, inRp, beta, params, t):
    '''
    Function to capture set up for parameterising and
    running deterministic prison model.
    '''
   
    # finish caclulating initial conditions
    incon = inE + inIu + inId + inM + inH + inRh + inRm + inRp
    inS = [params['Np'] - sum(incon)]
    incon = inS + incon 
    
    # set parameters 
    model.parameters = {'beta':beta}
    model.parameters = params
    
    # set initial conditions
    model.initial_values = (incon, t[0])
    
    # solve
    sol = model.integrate(t[1::])
    
    return t, sol, model

In [ ]:
def prison_dontexcludeM(ne=2, nu=2, nd=2, nm=2, nh=2):
    '''
    Template for seir compartmental structure which automatically 
    generates additional E and I compartments. 
    '''
    
    states = ['S']                               # susceptible 
    states += ['E%d' %(x+1) for x in range(ne)]  # incubating 
    states += ['Iu%d' %(x+1) for x in range(nu)] # infectious undetected 
    states += ['Id%d' %(x+1) for x in range(nd)] # infectious detected 
    states += ['M%d' %(x+1) for x in range(nm)]  # medical isolation
    states += ['H%d' %(x+1) for x in range(nh)]  # infectious household 
    states += ['Rh']                             # removed from hospital
    states += ['Rm']                             # removed from medical isolation 
    states += ['Rp']                             # undetected no longer infectious 

    # generate output to facilitate summing multiple compartments
    group = ['S']
    group += ['E' for x in range(ne)]
    group += ['Iu' for x in range(nu)]
    group += ['Id' for x in range(nd)]
    group += ['M' for x in range(nm)]
    group += ['H' for x in range(nh)]
    group += ['Rh']
    group += ['Rm']
    group += ['Rp']

    # model parameters
    parameters = ['beta',  # ro = beta/gamma= beta*th2
                  'ta',    # incubation period 
                  'tg',    # infectious period 
                  'td',    # mean time to detection for individauls that will be detected
                  'tm',    # mean time in 
                  'th',    # mean time in hospital
                  'kc',    # proportion of detected cases
                  'kh',    # proportion of hospitalised cases
                  'Np',    # total prison population
                  'ne',    # number of compartments for distributing incubation period 
                  'nu',    # number of compartments for distributing undetected infectious compartments
                  'nd',    # number of compartments for distributing detected infectious compartments
                  'nm',    # number of compartments for distributing medically isolated compartments
                  'nh'     # number of compartments for distributing hospitalised comparments
                 ]

    transitions = []

    ## infections
    # new infections generated from Iu compartments
    for z in range(nu):
        transitions += [Transition(origin='S', destination='E1', 
                        equation='beta*S*Iu%d/(Np-H1-H2-Rh)' %(z+1),
                        transition_type=TransitionType.T)]

    # new infections generated from Id compartments
    for z in range(nd):
        transitions += [Transition(origin='S', destination='E1', 
                        equation='beta*S*Id%d/(Np-H1-H2-Rh)' %(z+1),
                        transition_type=TransitionType.T)]

    ## incubating
    # generate transistions for E states
    if ne > 1:
        for x in range(ne-1):
            transitions += [Transition(origin='E%d' %(x+1), destination='E%d' %(x+2), 
                            equation='%d*E%d/ta' %(ne,x+1),
                            transition_type=TransitionType.T)]

    # transition from last E to Iu1        
    transitions += [Transition(origin='E%d' %ne, destination='Iu1', 
                    equation='(1-kc)*%d*E%d/ta' %(ne,ne),
                    transition_type=TransitionType.T)]       

    # transition from last E to Id1  
    transitions += [Transition(origin='E%d' %ne, destination='Id1', 
                    equation='kc*%d*E%d/ta' %(ne,ne),
                    transition_type=TransitionType.T)] 

    ## infectious undetected
    # generate transitions for Iu states
    if nu > 1:
        for y in range(nu-1):
            transitions += [Transition(origin='Iu%d' %(y+1), destination='Iu%d' %(y+2),
                            equation='%d*Iu%d/tg' %(nu,y+1),
                            transition_type=TransitionType.T)]

    # transition from last Iu to Rp        
    transitions += [Transition(origin='Iu%d' %nu, destination='Rp', 
                    equation='%d*Iu%d/tg' %(nu,nu),
                    transition_type=TransitionType.T)]   

    ## infectious detected
    # generate transitions for Id states
    if nd > 1:
        for y in range(nd-1):
            transitions += [Transition(origin='Id%d' %(y+1), destination='Id%d' %(y+2), 
                            equation='%d*Id%d/td' %(nd,y+1),
                            transition_type=TransitionType.T)]

    # transition from last Id to M1        
    transitions += [Transition(origin='Id%d' %nd, destination='M1', 
                    equation='(1-kh)*%d*Id%d/td' %(nd,nd),
                    transition_type=TransitionType.T)]   

    # transition from last Id to H1        
    transitions += [Transition(origin='Id%d' %nd, destination='H1', 
                    equation='kh*%d*Id%d/td' %(nd,nd),
                    transition_type=TransitionType.T)]   

    ## medical isolation
    # generate transitions for M states
    if nm > 1:
        for y in range(nm-1):
            transitions += [Transition(origin='M%d' %(y+1), destination='M%d' %(y+2),
                            equation='%d*M%d/tm' %(nm,y+1),
                            transition_type=TransitionType.T)]

    # transition from last M to Rp        
    transitions += [Transition(origin='M%d' %nm, destination='Rm', 
                    equation='%d*M%d/tm' %(nm,nm),
                    transition_type=TransitionType.T)]        

    ## hospital
    # generate transitions for H states
    if nh > 1:
        for y in range(nh-1):
            transitions += [Transition(origin='H%d' %(y+1), destination='H%d' %(y+2),
                            equation='%d*H%d/th' %(nh,y+1),
                            transition_type=TransitionType.T)]

    # transition from last H to Rh        
    transitions += [Transition(origin='H%d' %nh, destination='Rh', 
                    equation='%d*H%d/th' %(nh,nh),
                    transition_type=TransitionType.T)]        

    model = DeterministicOde(states, parameters, transition=transitions)
    
    model.parameters = {'ne':ne, 'nu':nu, 'nd':nd, 'nm':nm, 'nh':nh}

    return model, group

In [ ]:
# parameters
R0 = 3.0
ta = 6
tg = 4
td = 1
tm = 7
th = 22

## Unmitigated scenario

### Reproduction number and calculating transmission parameter 
The reproduction number is $R_0 = \beta (k_ct_d + (1-k_c)t_g)$.

- In the unmitigated scenario, $R_0$ = 3. 
- We assume that an unmitigated scenario means that all individuals, irrespective of their disease pathway, 
are infectious for the total infectiousness period ($t_g=4$). Hospitalisation pathway is still included ($t_d = t_g$), but no medical isolation occurs ($k_h=1$). 
- From this we determine the transmission parameter. 

In [ ]:
beta = R0/tg
print(beta)

This will be used as the baseline value for beta (is irrespective of population size).

- When there are no isolation procedures in place, to obtain an $R_0<1$, $\beta<1/t_g = 0.25$. 
This requires a regime changes to reduce the risk of transmission by 

In [ ]:
print('%.2f %%' %round((beta-1/tg)/beta*100,2))

In [ ]:
def proportions(sol, Np):
    total_infected_prop = round(sum(sol[-1,-3::]) / Np * 100,2)
    total_clinical_prop = sum(sol[-1,-3:-1])/ Np *100
    total_hospitalised_prop = sol[-1,-3] / Np * 100
    infected_hospitalised_prop = total_hospitalised_prop/total_infected_prop * 100
    
    print('%.2f %% total population infected' %(total_infected_prop))
    print('%.2f %% total population clinical ' %(total_clinical_prop))
    print('%.2f %% total population hopsitalised ' %(total_hospitalised_prop))
    print('%.2f %% infected population hospitalised' %(infected_hospitalised_prop))
    

In [ ]:
# generate model 
model_base, group = prison_dontexcludeM(ne=ne, nu=nu, nd=nd, nm=nm, nh=nh)

# calculate R0 from matrix
compartments = [x.name for x in model_base.state_list]
m, n = epi_analysis.disease_progression_matrices(model_base, compartments[1:(ne+nu+nd)+1])
epi_analysis.R0_from_matrix(m, n)

In [ ]:
model_base.parameters

In [ ]:
# parameters for baseline according to above specified values

pars_base = {'ta': ta,
             'tg': tg,
             'td': tg, # full infectious period
             'tm': tm, # no removal of cases from general population
             'th': th,
             'kc': 0.53, 
             'kh': 0.11, # all are hospitalised
             'Np': Np
            }

# initial conditions
inE = [1, 0] # seed outbreak with 1 initially infected person
inIu = [0, 0]
inId = [0, 0]
inM = [0, 0]
inH = [0, 0]
inRh = [0]
inRm = [0]
inRp = [0]

t = np.linspace(0, 200, 201)


In [ ]:
# run regime changes to no hospital structure
tbase, solbase, modbase = run_prison(model=model_base,
                                     inE=inE,
                                     inIu=inIu,
                                     inId=inId,
                                     inM=inM,
                                     inH=inH,
                                     inRh=inRh,
                                     inRm=inRm,
                                     inRp=inRp,
                                     beta=beta, 
                                     params=pars_base,
                                     t=t)

tbase075, solbase075, modbase075 = run_prison(model=model_base,
                                     inE=inE,
                                     inIu=inIu,
                                     inId=inId,
                                     inM=inM,
                                     inH=inH,
                                     inRh=inRh,
                                     inRm=inRm,
                                     inRp=inRp,
                                     beta=0.75*beta, 
                                     params=pars_base,
                                     t=t)
tbase050, solbase050, modbase050 = run_prison(model=model_base,
                                     inE=inE,
                                     inIu=inIu,
                                     inId=inId,
                                     inM=inM,
                                     inH=inH,
                                     inRh=inRh,
                                     inRm=inRm,
                                     inRp=inRp,
                                     beta=0.5*beta, 
                                     params=pars_base,
                                     t=t)
tbase025, solbase025, modbase025 = run_prison(model=model_base,
                                     inE=inE,
                                     inIu=inIu,
                                     inId=inId,
                                     inM=inM,
                                     inH=inH,
                                     inRh=inRh,
                                     inRm=inRm,
                                     inRp=inRp,
                                     beta=0.25*beta, 
                                     params=pars_base,
                                     t=t)

In [ ]:
proportions(solbase, pars_base['Np'])

In [ ]:
proportions(solbase075, pars_base['Np'])

In [ ]:
proportions(solbase050, pars_base['Np'])

In [ ]:
proportions(solbase025, pars_base['Np'])

In [ ]:
# determine index of compartments
Sind = [i for i, x in enumerate(group) if x == 'S']
Eind = [i for i, x in enumerate(group) if x == 'E']
Iuind = [i for i, x in enumerate(group) if x == 'Iu']
Idind = [i for i, x in enumerate(group) if x == 'Id']
Mind = [i for i, x in enumerate(group) if x == 'M']
Hind = [i for i, x in enumerate(group) if x == 'H']
Rhind = [i for i, x in enumerate(group) if x == 'Rh']
Rmind = [i for i, x in enumerate(group) if x == 'Rm']
Rpind = [i for i, x in enumerate(group) if x == 'Rp']

# plot
fig = plt.figure(figsize=[10,10])
ax = plt.subplot(1,1,1)
ax.plot(t, solbase[:,0], label='Susceptible')
ax.plot(t, np.sum(solbase[:,Eind], axis=1), label='Incubating') 
ax.plot(t, np.sum(solbase[:,Iuind] + solbase[:,Idind], axis=1), label='Infectious') 
ax.plot(t, np.sum(solbase[:,Mind]+solbase[:,Hind], axis=1), label='Isolated') 
ax.plot(t, solbase[:,-3]+solbase[:,-2], label='Recovered (isolated)')
ax.plot(t, solbase[:,-1], label='Recovered (prison)')
ax.set_ylabel('People')
ax.set_xlabel('Days since initial infection')
ax.legend(facecolor='white', frameon=False)

plt.savefig('pub_unmitigated_det.png', dpi=300)

In [ ]:
Rhind = [i for i, x in enumerate(group) if x == 'Rh']
Rmind = [i for i, x in enumerate(group) if x == 'Rm']
Rpind = [i for i, x in enumerate(group) if x == 'Rp']

inftotind = Rhind+Rmind+Rpind
infclin = Rhind+Rmind

In [ ]:
# plot infections and recovered

infind = Iuind + Idind
inftotind = Rhind+Rmind+Rpind

fig = plt.figure(figsize=[18,10])
plt.rcParams['font.size'] = 14
ax = plt.subplot(1,2,1)
ax.plot(tbase, np.sum(solbase[:,infind],axis=1), label='no reduction')
ax.plot(tbase075, np.sum(solbase075[:,infind],axis=1), label='25% reduction')
ax.plot(tbase050, np.sum(solbase050[:,infind],axis=1), label='50% reduction')
ax.plot(tbase025, np.sum(solbase025[:,infind],axis=1), label='75% reduction')
ax.set_ylabel('People')
np.sum(solbase075[:,infind],axis=1)
ax.legend(loc=1,frameon=False)
ax.set_xlabel('Days since initial infection')
ax.set_title('Infections')

ax = plt.subplot(1,2,2)
ax.plot(tbase, np.sum(solbase[:,inftotind],axis=1))
ax.plot(tbase, np.sum(solbase075[:,inftotind],axis=1))
ax.plot(tbase, np.sum(solbase050[:,inftotind],axis=1))
ax.plot(tbase, np.sum(solbase025[:,inftotind],axis=1))
ax.set_xlabel('Days since initial infection')
ax.set_title('Cumulative infections')


plt.savefig('figures/pub_det_regime_inf.png', dpi=300)

In [ ]:
proportions(solbase, Np)

## 2. Regime changes on top of hospitalisation and isolation

We are now assuming that isolation of clinical cases, and hospitalisation of severe cases is in place. 


Reducing the contact rate by 25%, 50%, and 75% corresponds to a reduction of $\beta$ by these proportions.

In [ ]:
# adjust population to remove shielded people
shielded = 0.012
Ns = round(Np*shielded)

pars = {'ta': ta,
        'tg': tg,
        'td': td, 
        'tm': tm, 
        'th': th,
        'kc': 0.53, 
        'kh': 0.11, 
        'Np': Np-Ns
       }

ty = np.linspace(0, 365, 366)

model, group = prison(ne=ne, nu=nu, nd=nd, nm=nm, nh=nh)

### Reproduction number and transmission parameter
- When there are isolation procedures in place, assuming other contact and risk of transmission has not changed, the reproduction number can now take into account the reduced infectious time of people who are detected. 

$R_0 = \beta(k_ct_d + (1-k_c)t_g)$

In [ ]:
R0_mit= beta*(pars['kc']*pars['td'] + (1-pars['kc'])*pars['tg'])
R0_reduction = (R0-R0_mit)/R0*100
print('The reproduction number is now %.2f. This change has reduced the reproduction number by %.2f %%.' 
      %(R0_mit, R0_reduction))

print('A regime change which reduces the risk of transmission by at least %.2f %% will push the reproduction number below 1.' 
      %((R0_mit-1)/R0_mit*100))

In [ ]:
# solutions for regime changes

t100, sol100, modbase100 = run_prison(model=model, inE=inE, inIu=inIu, inId=inId, inM=inM, inH=inH, 
                                      inRh=inRh, inRm=inRm, inRp=inRp, beta=beta, params=pars, t=ty)
proportions(sol100, pars['Np'])

In [ ]:
t075, sol075, modbase075 = run_prison(model=model, inE=inE, inIu=inIu, inId=inId, inM=inM, inH=inH, 
                                      inRh=inRh, inRm=inRm, inRp=inRp, beta=0.75*beta, params=pars, t=ty)
proportions(sol075, pars['Np'])

In [ ]:
t050, sol050, modbase050 = run_prison(model=model, inE=inE, inIu=inIu, inId=inId, inM=inM, inH=inH, 
                                      inRh=inRh, inRm=inRm, inRp=inRp, beta=0.5*beta, params=pars, t=ty)
proportions(sol050, pars['Np'])

In [ ]:
t025, sol025, modbase025 = run_prison(model=model, inE=inE, inIu=inIu, inId=inId, inM=inM, inH=inH, 
                                      inRh=inRh, inRm=inRm, inRp=inRp, beta=0.25*beta, params=pars, t=ty)
proportions(sol025, pars['Np'])

In [ ]:
# plot
fig = plt.figure(figsize=[18,16])
ax = plt.subplot(3,3,1)

ax.plot(t100, sol100[:,0], label=r'$\beta$')
ax.plot(t075, sol075[:,0], label=r'$0.75\beta$')
ax.set_ylabel('People')
ax.set_title('Susceptible')

ax = plt.subplot(3,3,2)
ax.plot(t100, np.sum(sol100[:,Eind], axis=1)) 
ax.plot(t075, np.sum(sol075[:,Eind], axis=1)) 
ax.set_title('Incubating')

ax = plt.subplot(3,3,3)
ax.plot(t100, np.sum(sol100[:,Iuind], axis=1), label=r'$\beta$') 
ax.plot(t075, np.sum(sol075[:,Iuind], axis=1), label=r'$0.75\beta$')
ax.legend(loc=1, frameon=False)
ax.set_title('Infectious - not clinical')

ax = plt.subplot(3,3,4)
ax.plot(t100, np.sum(sol100[:,Idind], axis=1)) 
ax.plot(t075, np.sum(sol075[:,Idind], axis=1)) 
ax.set_ylabel('People')
ax.set_title('Infectious - clinical')

ax = plt.subplot(3,3,5)
ax.plot(t100, np.sum(sol100[:,Mind], axis=1)) 
ax.plot(t075, np.sum(sol075[:,Mind], axis=1)) 
ax.set_title('Medical isolation')

ax = plt.subplot(3,3,6)
ax.plot(t100, np.sum(sol100[:,Hind], axis=1)) 
ax.plot(t075, np.sum(sol075[:,Hind], axis=1)) 
ax.set_title('Hospitalised')

ax = plt.subplot(3,3,7)
ax.plot(t100, np.sum(sol100[:,Rhind], axis=1)) 
ax.plot(t075, np.sum(sol075[:,Rhind], axis=1)) 
ax.set_ylabel('People')
ax.set_xlabel('Days since initial seeding')
ax.set_title('Cumulative hospitalised')

ax = plt.subplot(3,3,8)
ax.plot(t100, np.sum(sol100[:,Rmind], axis=1)) 
ax.plot(t075, np.sum(sol075[:,Rmind], axis=1)) 
ax.set_xlabel('Days since initial seeding')
ax.set_title('Cumulative medically isolated')

ax = plt.subplot(3,3,9) 
ax.plot(t100, np.sum(sol100[:,Rpind], axis=1)) 
ax.plot(t075, np.sum(sol075[:,Rpind], axis=1))  
ax.set_xlabel('Days since initial seeding')
ax.set_title('Cumulative undetected infected')

plt.savefig('figures/pub_det_reducedbeta_1.png', dpi=300)

In [ ]:
# plot
fig = plt.figure(figsize=[18,16])
ax = plt.subplot(3,3,1)
ax.plot(t050, sol050[:,0], label=r'$0.5\beta$', color='green')
ax.plot(t025, sol025[:,0], label=r'$0.25\beta$', color='red')
ax.set_ylabel('People')
ax.set_title('Susceptible')

ax = plt.subplot(3,3,2)
ax.plot(t050, np.sum(sol050[:,Eind], axis=1), color='green') 
ax.plot(t025, np.sum(sol025[:,Eind], axis=1), color='red') 
ax.set_title('Incubating')

ax = plt.subplot(3,3,3)
ax.plot(t050, np.sum(sol050[:,Iuind], axis=1), label=r'$0.5\beta$',  color='green') 
ax.plot(t025, np.sum(sol025[:,Iuind], axis=1),  label=r'$0.25\beta$', color='red') 
ax.set_title('Infectious - not clinical')

ax = plt.subplot(3,3,4)
ax.plot(t050, np.sum(sol050[:,Idind], axis=1), color='green') 
ax.plot(t025, np.sum(sol025[:,Idind], axis=1), color='red') 
ax.set_ylabel('People')
ax.set_title('Infectious - clinical')

ax = plt.subplot(3,3,5)
ax.plot(t050, np.sum(sol050[:,Mind], axis=1), color='green') 
ax.plot(t025, np.sum(sol025[:,Mind], axis=1), color='red') 
ax.set_title('Medical isolation')

ax = plt.subplot(3,3,6) 
ax.plot(t050, np.sum(sol050[:,Hind], axis=1), color='green') 
ax.plot(t025, np.sum(sol025[:,Hind], axis=1), color='red') 
ax.set_title('Hospitalised')

ax = plt.subplot(3,3,7)
ax.plot(t050, np.sum(sol050[:,Rhind], axis=1), color='green') 
ax.plot(t025, np.sum(sol025[:,Rhind], axis=1), color='red') 
ax.set_ylabel('People')
ax.set_xlabel('Days since initial seeding')
ax.set_title('Cumulative hospitalised')

ax = plt.subplot(3,3,8)
ax.plot(t050, np.sum(sol050[:,Rmind], axis=1), color='green') 
ax.plot(t025, np.sum(sol025[:,Rmind], axis=1), color='red') 
ax.set_xlabel('Days since initial seeding')
ax.set_title('Cumulative medically isolated')

ax = plt.subplot(3,3,9)
ax.plot(t050, np.sum(sol050[:,Rpind], axis=1), color='green') 
ax.plot(t025, np.sum(sol025[:,Rpind], axis=1), color='red')  
ax.set_xlabel('Days since initial seeding')
ax.set_title('Cumulative undetected infected')


plt.savefig('figures/pub_det_reducedbeta_2.png', dpi=300)

In [ ]:
totinfind = Rhind+Rmind+Rpind
totclinind = Rhind+Rmind

# plot infected, clinical, hosp for no reduction, 25, 50, 75% reduction
fig = plt.figure(figsize=[18,16])
ax = plt.subplot(2,3,1)

ax.plot(t100, np.sum(sol100[:,totinfind], axis=1), label='no reduction') 
ax.plot(t075, np.sum(sol075[:,totinfind], axis=1), label='25% reduction') 
ax.set_ylabel('People')
ax.set_title('Total infected')

ax = plt.subplot(2,3,2)
ax.plot(t100, np.sum(sol100[:,totclinind], axis=1)) 
ax.plot(t075, np.sum(sol075[:,totclinind], axis=1)) 
ax.set_title('Total clinically infected')

ax = plt.subplot(2,3,3)
ax.plot(t100, np.sum(sol100[:,Rhind], axis=1), label='no reduction') 
ax.plot(t075, np.sum(sol075[:,Rhind], axis=1), label='25% reduction')  
ax.set_title('Total hospitalised')
ax.legend(loc=4, frameon=False, edgecolor=None, facecolor='white', shadow=False)

ax = plt.subplot(2,3,4)
ax.plot(t100, np.sum(sol050[:,totinfind], axis=1), color='green', label='50% reduction') 
ax.plot(t075, np.sum(sol025[:,totinfind], axis=1), color='red', label='75% reduction')
ax.set_ylabel('People')
ax.set_title('Total infected')
ax.set_xlabel('Days since initial seeding')

ax = plt.subplot(2,3,5)
ax.plot(t100, np.sum(sol050[:,totclinind], axis=1), color='green') 
ax.plot(t075, np.sum(sol025[:,totclinind], axis=1), color='red') 
ax.set_title('Total clinically infected')
ax.set_xlabel('Days since initial seeding')

ax = plt.subplot(2,3,6)
ax.plot(t100, np.sum(sol050[:,Rhind], axis=1), color='green', label='50% reduction') 
ax.plot(t075, np.sum(sol025[:,Rhind], axis=1), color='red', label='75% reduction')  
ax.set_title('Total hospitalised')
ax.set_xlabel('Days since initial seeding')
ax.legend(loc=7, frameon=False, edgecolor=None, facecolor='white')

plt.savefig('figures/pub_det_reducedbeta_totals.png', dpi=300)


The regime change which reduces $\beta$ by 50% and 75% is sufficient to bring the epidemic under control.

Now, run these stochastically.

In [ ]:
def stocrun(model, inE, inIu, inId, inM, inH, inRh, inRm, inRp, beta, params, t, x, parallel=False):
    '''
    Stochastic simulations of model. This does not contain the cumulative infections.
    '''
    
    # set up stochastic system
    smod = SimulateOde(state=model.state_list,
                       param=model.param_list,
                       transition=model.transition_list)
    
    # initial conditions
    incon = inE + inIu + inId + inM + inH + inRh + inRm + inRp
    inS = [params['Np'] - sum(incon)]
    incon = inS + incon
    
    smod.initial_values = (incon, t[0])
    smod.parameters = {'beta':beta}
    smod.parameters=params
    
    simX, simT = smod.simulate_jump(t, x, full_output=True, parallel=False)
    
    return simX, simT

def stoc_plot_quants(simX, simT, Sind, Eind, Iuind, Idind, Mind, Hind, Rhind, Rmind, Rpind, x,
                     min_case=5, figdir='figures/fig'):
    '''Plot quantiles of solutions.'''
    
    sus = np.array([sim_res[:,Sind] for sim_res in simX])
    exp = np.sum(np.array([sim_res[:,Eind] for sim_res in simX]), axis=2)
    inu = np.sum(np.array([sim_res[:,Iuind] for sim_res in simX]), axis=2)
    ind = np.sum(np.array([sim_res[:,Idind] for sim_res in simX]), axis=2)
    med = np.sum(np.array([sim_res[:,Mind] for sim_res in simX]), axis=2)
    hos = np.sum(np.array([sim_res[:,Hind] for sim_res in simX]), axis=2)
    rho = np.array([sim_res[:,Rhind] for sim_res in simX]) 
    rmi = np.array([sim_res[:,Rmind] for sim_res in simX])
    rpr = np.array([sim_res[:,Rpind] for sim_res in simX])
    
    
    # condition for removing simulations when extinction occurs
    # returns True False list
    cond = rpr[:,-1] + rho[:,-1] > min_case
    cond = cond[:,0]   

    sus_quant = np.transpose(np.quantile(sus[cond,:], (0.025, 0.25, 0.5, 0.75, 0.975), axis=0))[0]
    exp_quant = np.transpose(np.quantile(exp[cond,:], (0.025, 0.25, 0.5, 0.75, 0.975), axis=0))
    inu_quant = np.transpose(np.quantile(inu[cond,:], (0.025, 0.25, 0.5, 0.75, 0.975), axis=0))
    ind_quant = np.transpose(np.quantile(ind[cond,:], (0.025, 0.25, 0.5, 0.75, 0.975), axis=0))
    med_quant = np.transpose(np.quantile(med[cond,:], (0.025, 0.25, 0.5, 0.75, 0.975), axis=0))
    hos_quant = np.transpose(np.quantile(hos[cond,:], (0.025, 0.25, 0.5, 0.75, 0.975), axis=0))
    rho_quant = np.transpose(np.quantile(rho[cond,:], (0.025, 0.25, 0.5, 0.75, 0.975), axis=0))[0]
    rmi_quant = np.transpose(np.quantile(rmi[cond,:], (0.025, 0.25, 0.5, 0.75, 0.975), axis=0))[0]
    rpr_quant = np.transpose(np.quantile(rpr[cond,:], (0.025, 0.25, 0.5, 0.75, 0.975), axis=0))[0]
    
    

    fig = plt.figure(figsize=[18,16])

    ax = plt.subplot(3,3,1)
    ax.plot(simT, sus_quant[:,2], label='median', color='black')
    ax.fill_between(simT, sus_quant[:,0], sus_quant[:,4], color='blue',
                    alpha=0.1, label='2.5 - 97.5 quantile')
    ax.fill_between(simT, sus_quant[:,1], sus_quant[:,3], color='blue',
                    alpha = 0.2, label='25 - 75 quantile')
    ax.set_ylabel('People')
    ax.set_title('Susceptible')

    ax = plt.subplot(3,3,2)
    ax.plot(simT, exp_quant[:,2], label='median', color='black')
    ax.fill_between(simT, exp_quant[:,0], exp_quant[:,4], color='blue',
                    alpha = 0.1, label='2.5 - 97.5 quantile')
    ax.fill_between(simT, exp_quant[:,1], exp_quant[:,3], color='blue',
                    alpha = 0.2, label='25 - 75 quantile')
    ax.set_title('Incubating')

    ax = plt.subplot(3,3,3)
    ax.plot(simT, inu_quant[:,2], label='median', color='black')
    ax.fill_between(simT, inu_quant[:,0], inu_quant[:,4], color='blue',
                    alpha=0.1, label='2.5 - 97.5 quantile')
    ax.fill_between(simT, inu_quant[:,1], inu_quant[:,3], color='blue',
                    alpha=0.2, label='25 - 75 quantile')
    ax.set_title('Infectious - not clinical')
    
    ax = plt.subplot(3,3,4)
    ax.plot(simT, ind_quant[:,2], label='median', color='black')
    ax.fill_between(simT, ind_quant[:,0], ind_quant[:,4], color='blue',
                    alpha=0.1, label='2.5 - 97.5 quantile')
    ax.fill_between(simT, ind_quant[:,1], ind_quant[:,3], color='blue',
                    alpha=0.2, label='25 - 75 quantile')
    ax.set_ylabel('People')
    ax.set_title('Infectious - clinical')

    ax = plt.subplot(3,3,5)
    ax.plot(simT, med_quant[:,2], label='median', color='black')
    ax.fill_between(simT, med_quant[:,0], med_quant[:,4], color='blue',
                    alpha=0.1, label='2.5 - 97.5 quantile')
    ax.fill_between(simT, med_quant[:,1], med_quant[:,3], color='blue',
                    alpha=0.2, label='25 - 75 quantile')
    ax.set_title('Medical isolation')
    
    ax = plt.subplot(3,3,6)
    ax.plot(simT, hos_quant[:,2], label='median', color='black')
    ax.fill_between(simT, hos_quant[:,0], hos_quant[:,4], color='blue',
                    alpha=0.1, label='2.5 - 97.5 quantile')
    ax.fill_between(simT, hos_quant[:,1], hos_quant[:,3], color='blue',
                    alpha=0.2, label='25 - 75 quantile')
    ax.set_title('Hospitalised')
    
    ax = plt.subplot(3,3,7)
    ax.plot(simT, rho_quant[:,2], label='median', color='black')
    ax.fill_between(simT, rho_quant[:,0], rho_quant[:,4], color='blue',
                    alpha=0.1, label='2.5 - 97.5 quantile')
    ax.fill_between(simT, rho_quant[:,1], rho_quant[:,3], color='blue',
                    alpha=0.2, label='25 - 75 quantile')
    ax.set_ylabel('People')
    ax.set_xlabel('Days since initial seeding')
    ax.set_title('Cumulative hospitalised')

    ax = plt.subplot(3,3,8)
    ax.plot(simT, rmi_quant[:,2], label='median', color='black')
    ax.fill_between(simT, rmi_quant[:,0], rmi_quant[:,4], color='blue',
                    alpha=0.1, label='2.5 - 97.5 quantile')
    ax.fill_between(simT, rmi_quant[:,1], rmi_quant[:,3], color='blue',
                    alpha=0.2, label='25 - 75 quantile')
    ax.set_xlabel('Days since initial seeding')
    ax.set_title('Cumulative medically isolated')
    
    ax = plt.subplot(3,3,9)
    ax.plot(simT, rpr_quant[:,2], label='median', color='black')
    ax.fill_between(simT, rpr_quant[:,0], rpr_quant[:,4], color='blue',
                    alpha=0.1, label='2.5 - 97.5 quantile')
    ax.fill_between(simT, rpr_quant[:,1], rpr_quant[:,3], color='blue',
                    alpha=0.2, label='25 - 75 quantile')
    ax.legend(loc=4, facecolor='white', frameon=False)
    ax.set_xlabel('Days since initial seeding')
    ax.set_title('Cumulative undetected infectected')   
    
    
    plt.savefig(figdir, dpi=300)

    per_cond =(x-rpr[cond,:].shape[0])/rpr.shape[0]*100

    print('%.2f %% of the %d realisations result in fewer than %d cases. These have not been plotted.' %(per_cond, x, min_case))
    
    inf_total_quant = sum([rho_quant[-1:,:], rmi_quant[-1,:], rpr_quant[-1,:]])/Np
    inf_clinical_quant = sum([rho_quant[-1:,:], rmi_quant[-1,:]])/Np
    inf_hospital_quant = rho_quant[-1:,:]/Np
    attack_quants = [inf_total_quant, inf_clinical_quant, inf_hospital_quant]
    
    return cond, attack_quants 

def hist_plot(data, figdir):
    
    rho_fin = np.array([sim_res[-1,Rhind] for sim_res in data]) 
    rmi_fin = np.array([sim_res[-1,Rmind] for sim_res in data])
    rpr_fin = np.array([sim_res[-1,Rpind] for sim_res in data])

    total_inf = sum([rho_fin, rmi_fin, rpr_fin])/Np 
    total_det = sum([rho_fin, rmi_fin])/Np
    total_hosp = rho_fin/Np

    bins = 20

    fig = plt.figure(figsize=[15,8])
    plt.rcParams.update({'font.size': 14})
    ax = plt.subplot(1,3,1)
    ax.hist(total_inf, bins=bins, density=True)
    ax.set_xlabel('Proportion infected (total)')
    ax.set_ylabel('Frequency')

    ax = plt.subplot(1,3,2)
    ax.hist(total_det, bins=bins, density=True)
    ax.set_xlabel('Propotion infected (detected)')

    ax = plt.subplot(1,3,3)
    ax.hist(total_hosp, bins=bins, density=True)
    ax.set_xlabel('Proportion hospitalised')

    plt.savefig(figdir, dpi=300)

In [ ]:
model.state_list

In [ ]:
# 100%
x = 10000
simX100, simT100 = stocrun(model=model, inE=inE, inIu=inIu, inId=inId, inM=inM,
                           inH=inH, inRh=inRh, inRm=inRm, inRp=inRp,
                           beta=beta, params=pars, t=ty, x=x, parallel=True)

cond100, quants100 = stoc_plot_quants(simX100, simT100, Sind, Eind, Iuind, Idind,
                                      Mind, Hind, Rhind, Rmind, Rpind, x,
                                      min_case=5, figdir='figures/pub_stoc_int_beta.png')


In [ ]:
quants100

In [ ]:
# 75%
x=10000
simX075, simT075 = stocrun(model=model, inE=inE, inIu=inIu, inId=inId, inM=inM,
                           inH=inH, inRh=inRh, inRm=inRm, inRp=inRp,
                           beta=0.75*beta, params=pars, t=ty, x=x, parallel=True)

cond075, quants075 = stoc_plot_quants(simX075, simT075, Sind, Eind, Iuind, Idind,
                                      Mind, Hind, Rhind, Rmind, Rpind, x,
                           min_case=5, figdir='figures/pub_stoc_int_075beta.png')

In [ ]:
cond075_0, quants075_0 = stoc_plot_quants(simX075, simT075, Sind, Eind, Iuind, Idind,
                                          Mind, Hind, Rhind, Rmind, Rpind, x,
                           min_case=0, figdir='figures/pub_stoc_int_075beta_all.png')

In [ ]:
hist_plot(simX075, figdir='figures/pub_075beta_hist.png')

In [ ]:
quants075

In [ ]:
# 50%
simX050, simT050 = stocrun(model=model, inE=inE, inIu=inIu, inId=inId, inM=inM,
                           inH=inH, inRh=inRh, inRm=inRm, inRp=inRp,
                           beta=0.5*beta, params=pars, t=ty, x=x, parallel=True)

cond050, quants050 = stoc_plot_quants(simX050, simT050, Sind, Eind, Iuind, Idind,
                                      Mind, Hind, Rhind, Rmind, Rpind, x,
                                      min_case=5, figdir='figures/pub_stoc_int_050beta.png')


In [ ]:
cond050_0, quants050_0 = stoc_plot_quants(simX050, simT050, Sind, Eind, Iuind, Idind,
                                          Mind, Hind, Rhind, Rmind, Rpind, x,
                                          min_case=0, figdir='figures/pub_stoc_int_050beta_all.png')


In [ ]:
hist_plot(simX050, figdir='figures/pub_050beta_hist.png')

In [ ]:
quants050

In [ ]:
# 25%
simX025, simT025 = stocrun(model=model, inE=inE, inIu=inIu, inId=inId, inM=inM,
                           inH=inH, inRh=inRh, inRm=inRm, inRp=inRp,
                           beta=0.25*beta, params=pars, t=ty, x=x, parallel=True)

cond025, quants025 = stoc_plot_quants(simX025, simT025, Sind, Eind, Iuind, Idind,
                                      Mind, Hind, Rhind, Rmind, Rpind, x,
                                      min_case=5, figdir='figures/pub_stoc_int_025beta.png')

In [ ]:
cond025_0, quants025_0 = stoc_plot_quants(simX025, simT025, Sind, Eind, Iuind, Idind,
                                          Mind, Hind, Rpind, Rmind, Rhind, x,
                                          min_case=0, figdir='figures/pub_stoc_ind_025beta_all.png')

In [ ]:
hist_plot(simX025, figdir='figures/pub_025beta_hist.png')

In [ ]:
quants025